<a href="https://colab.research.google.com/github/Sayan3sarkar/ANN-Churn-Rate-Keras/blob/master/ANN_Churn_Rate_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Description

A Europe based bank has been noticing surprisingly high <a href="https://en.wikipedia.org/wiki/Churn_rate">churn rates</a> .The bank has prepared a spreadsheet having various details of each of it's customers over the past 6 months and wants to analyse,using this dataset, why the customers are ending their association with the bank.

We prepare an <b>Aritificial Neural Network(ANN)</b> using <a href="http://keras.io/">Keras API</a> to predict which customers are at higher chance of leaving the bank, and present that information to the banl itself so that it can interact with those customers to find their exact reasons for leaving

# Import necessary libraries 

In [0]:
import numpy as np
import pandas as pd

# Loading the Dataset and splitting into dependent and independent variables

<h3> Upload Dataset </h3>

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling (1).csv


In [0]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [0]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Here features such as $Row number, CustomerId, Surname$ do not help in deciding whether the customer will be associated with the bank or not. So we don't consider these in our feature matirx

In [0]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [0]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [0]:
y

array([1, 0, 1, ..., 1, 1, 0])

 From the above output cells, we clearly find that <b>Geography</b> and <b>Gender</b> are categorical variables. So encoding has to be done to convert the categorical variables to numerical values. We use <b>LabelEncoder</b> to perform this. Since,the dependent variable <b>y</b> is binary, we do not need to perform label encoding on it

In [0]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X1 = LabelEncoder()
X[:, 1] = labelencoder_X1.fit_transform(X[:, 1])
labelencoder_X2 = LabelEncoder()
X[:, 2] = labelencoder_X2.fit_transform(X[:, 2])

What we can see that there is no relational order between our categorical variables, so we need to perform <b>onehot encoding</b> to create dummy variables in rder to enforce relational order. We apply one hot encoding to only <b>Geography</b> as it has 3 categories ,namely <b>France, Germany</b> and <b>Spain</b>. <b>Gender</b> has only 2 categories.

In [0]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [1]) #Since we create dummy variables only for index 1, i.e, Geography
X = onehotencoder.fit_transform(X).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
X = X[:, 1:]# there are 3 dummy variables corresponding to 3 countries. We delete one dummy variable, the first coloumn to avoid falling into dummy variable trap

In [0]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

## Splitting dataset into training and test set. We allot a 80-20 ratio for training to test set size

In [0]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [0]:
X_test

array([[ 1.75486502, -0.57369368, -0.55204276, ...,  0.64259497,
         0.9687384 ,  1.61085707],
       [-0.5698444 , -0.57369368, -1.31490297, ...,  0.64259497,
        -1.03227043,  0.49587037],
       [-0.5698444 ,  1.74309049,  0.57162971, ...,  0.64259497,
         0.9687384 , -0.42478674],
       ...,
       [-0.5698444 ,  1.74309049, -0.74791227, ...,  0.64259497,
        -1.03227043,  0.71888467],
       [ 1.75486502, -0.57369368, -0.00566991, ...,  0.64259497,
         0.9687384 , -1.54507805],
       [ 1.75486502, -0.57369368, -0.79945688, ...,  0.64259497,
        -1.03227043,  1.61255917]])

# Making the ANN

## Importing Keras and other requisite modules

In [0]:
import keras

In [0]:
from keras.models import Sequential #For initializing ANN

In [0]:
from keras.layers import Dense # for adding layers to ANN
#from keras.layers import Dropout

## Initializing ANN

In [0]:
classifier =  Sequential()

In [0]:
#Adding first hidden layer
classifier.add(Dense(units = 6, input_dim = 11, activation = 'relu', kernel_initializer = 'uniform'))
#classifier.add(Dropout(p = 0.1))
#Adding Second hidden layer
classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer = 'uniform'))
#classifier.add(Dropout(p = 0.1))
#Ading output layer
classifier.add(Dense(units = 1, activation= 'sigmoid', kernel_initializer = 'uniform'))

In [0]:
#Compiling ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Fitting ANN to training set

In [0]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4958 - acc: 0.7957
Epoch 2/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4299 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 98us/step - loss: 0.4257 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 85us/step - loss: 0.4217 - acc: 0.7976
Epoch 5/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4188 - acc: 0.8221
Epoch 6/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.4163 - acc: 0.8252
Epoch 7/100
8000/8000 [==============================] - 1s 85us/step - loss: 0.4149 - acc: 0.8290
Epoch 8/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4132 - acc: 0.8307
Epoch 9/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4120 - acc: 0.8312
Epoch 10/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4109 - acc: 0.8316
Epoch 1

# Making predictions and evaluating the model

## Predicition on test set

In [0]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

## Confusion Matrix

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1536,   59],
       [ 256,  149]])

## From above confusion matrix we can clearly detemine that our prediction accuracy was ((1536+149)/2000)* 100 = 84.25%

We now test accuracy by giving details of a single customer into our ANN. Cutomer Details are as follows:

<b>Geography: France</b>

<b>Credit Score: 600</b>

<b>Gender: Male</b>

<b>Age: 40 years old</b>

<b>Tenure: 3 years</b>

<b>Balance: 60000</b>

<b>Number of Products: 2</b>

<b>Does this customer have a credit card ? Yes</b>

<b>Is this customer an Active Member: Yes</b>

<b>Estimated Salary: 50000</b>

In [0]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [0]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

### On comparing <i>X</i> and <i>dataset</i> , we find that <i>France</i> corresponds to 0,0. <i>Spain</i> corresponds to 0,1. <i>Germany</i> corresponds to 1,0. Also  <i>Gender</i>, 0 is <i>female</i> and 1 is <i>male</i> 

In [0]:
new_pred = classifier.predict(sc.transform(np.array([[0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_pred = (new_pred > 0.5)
new_pred

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[False]])

# Evaluating the ANN using k-fold cross validation

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
def create_ANN():
    model = Sequential()
    model.add(Dense(units = 6, input_dim = 11, activation = 'relu', kernel_initializer = 'uniform'))
    model.add(Dense(units = 6, activation = 'relu', kernel_initializer = 'uniform'))
    model.add(Dense(units = 1, activation= 'sigmoid', kernel_initializer = 'uniform'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [0]:
classifier1 = KerasClassifier(build_fn = create_ANN, batch_size = 10, epochs = 100)

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
accuracies = cross_val_score(classifier1, X_train, y_train, cv = 10, n_jobs = -1)

In [0]:
accuracies.mean()

0.8502499943599104

In [0]:
accuracies.std()

0.013984365291565616

# The above 2 cells clearly indicate that our model's accuracy is 85.02% on an average, and it has a variance of 13.98%